<hr>

 *PART II: Data Analysis* 

<a id='resnet'></a>
# Advanced Modeling

In this phase we will be exploring the image dataset in depth words words
In the Exploratory Data Analysis (EDA) phase of this project, a key focus is to examine the dataset in depth for any potential biases that could mistakenly be interpreted as distinguishing features by the machine learning models later on. This includes the orientation of the leaves as well as the color composition of the backround which makes up the majority of the pixels in any given image.


**Objectives:**

- Explore the dataset  
- Analyze color composition  

### Set up

In [2]:
## IMPORT LIBRARIES ##

import numpy as np #numpy
import pandas as pd # pandas
from matplotlib import pyplot as plt # matplot library
import seaborn as sns

In [3]:
## IMPORT LIBRARIES ##

import os  
from matplotlib import image as mpimg
from random import randint
from PIL import Image
from skimage.io import imread, imshow
import warnings 
warnings.filterwarnings('ignore')

##########

In [4]:
# IMPORT LIBRARIES

import tensorflow as tf
import keras 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

---

## CNN using Keras, Tensorflow

> **Process outline:**
>
>> **STEP 1:**
>> Load the data as a tensor
>> 
>> **STEP 2:** Set up a CNN Sequential Model
>> 
>> **STEP 3:** Compile Settings
>> 
>> **STEP 4:** Train the model
>> 
>> **STEP 5:** Evaluate the model
>> 

**STEP 1: Load the data as a tensor**  

The function we'll be using to load the dataset is `keras.utils.image_dataset_from_directory` and it generates a `tf.data.Dataset` object from an image files.  

These loading utilites can be combined with preprocessing layers to futher transform our input dataset before training. 

In [6]:
# Set the parameters for the split
validation_split = 0.2
seed = 123

# Load training data
train_images = tf.keras.utils.image_dataset_from_directory(
    'dataset/training',
    label_mode = "int",
    color_mode = "rgb",
    image_size = (600, 800),
    batch_size = 32,
    shuffle = False,
    seed = seed,
    validation_split = validation_split,
    subset = "training",  # Specify this is the training subset
    interpolation = "bilinear",
    follow_links = False,
    crop_to_aspect_ratio = False,
    pad_to_aspect_ratio = False,
    data_format = None,
    verbose = True
)

# Load validation data
validation_images = tf.keras.utils.image_dataset_from_directory(
    'dataset/training',
    label_mode = "int",
    color_mode = "rgb",
    image_size = (600, 800),
    batch_size = 32,
    shuffle = False,
    seed = seed,
    validation_split = validation_split,
    subset = "validation",  # Specify this is the validation subset
    interpolation = "bilinear",
    follow_links = False,
    crop_to_aspect_ratio = False,
    pad_to_aspect_ratio = False,
    data_format = None,
    verbose = False
)

# Load test data
test_images = keras.utils.image_dataset_from_directory(
    'dataset/testing',
    label_mode = "int",
    class_names = None,
    color_mode = "rgb",
    image_size = (600,800),
    batch_size = 32,
    shuffle = False,
    seed = seed,
    validation_split = None,
    subset = None,
    interpolation = "bilinear",
    data_format = None,
    verbose = True
)

Found 2720 files belonging to 3 classes.
Using 2176 files for training.
Using 544 files for validation.
Found 680 files belonging to 3 classes.


**STEP 2: Set up a CNN Sequential Model**

A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor. A Sequential model is not appropriate when:

- the model has multiple inputs or multiple outputs
- Any of the layers has multiple inputs or multiple outputs

We can create a Sequential model incrementally via the `add()` method:

In [7]:
# Create a Sequential Model

CNN_model = Sequential()

# Create simple CNN model architecture with Pooling for dimensionality reduction 
# and Dropout to reduce overfitting
CNN_model.add(Conv2D(32, kernel_size=(3, 3), activation = 'relu', input_shape = (600, 800, 3)))
CNN_model.add(MaxPooling2D(pool_size=(2, 2)))

CNN_model.add(Conv2D(64, (3, 3), activation='relu'))
CNN_model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output of our convolutional layers
CNN_model.add(Flatten())

# Add dense layers
CNN_model.add(Dense(128, activation='relu'))
CNN_model.add(Dense(64, activation='relu'))
CNN_model.add(Dense(3, activation='softmax'))

# Print out a summary of the network
CNN_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 598, 798, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 299, 399, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 297, 397, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 148, 198, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1875456)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │   240,058,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 240,086,339 (915.86 MB)

 Trainable params: 240,086,339 (915.86 MB)

 Non-trainable params: 0 (0.00 B)

**STEP 3: Compile Settings**


In the code below, the CNN model is being compiled with the specific configurations that determine how the model will be trained. Three main components that are specified during the compilation are the following:

1. **Loss Function:** The loss parameter is set to `sparse_categorical_crossentropy`. This loss function is commonly used for classification problems where the classes are mutually exclusive, meaning each data point belongs to exactly one class.

2. **Optimizer:** The optimizer parameter is set to `Adam`. Adam is an optimization algorithm that adjusts the weights of the network to minimize the loss function.
   
4. **Metrics:** The metrics parameter includes `accuracy`. This tells the model to track the accuracy during training and validation. Accuracy is a common metric for classification tasks and provides a straightforward interpretation by expressing the proportion of inputs that were correctly classified.



In [8]:
# Compile the model with the desired loss function, optimizer, and metric(s) to track
CNN_model.compile(loss = 'sparse_categorical_crossentropy',
                  optimizer = 'Adam',
                  metrics = ['accuracy'])

**STEP 4: Train the Model**

Now we can train the model using our training dataset. `epochs` refers to the number 

In [9]:
# Fit the model using the training and validation datasets
CNN_model.fit(
    train_images,
    epochs=5,
    verbose=1,
    validation_data=validation_images  # Specify the validation dataset here
)

Epoch 1/5
68/68 ━━━━━━━━━━━━━━━━━━━━ 268s 4s/step - accuracy: 0.4288 - loss: 3461.5281 - val_accuracy: 0.4320 - val_loss: 1.0446
Epoch 2/5
68/68 ━━━━━━━━━━━━━━━━━━━━ 262s 4s/step - accuracy: 0.4780 - loss: 1.0015 - val_accuracy: 0.4816 - val_loss: 1.0246
Epoch 3/5
68/68 ━━━━━━━━━━━━━━━━━━━━ 259s 4s/step - accuracy: 0.5942 - loss: 0.7999 - val_accuracy: 0.5846 - val_loss: 1.1771
Epoch 4/5
68/68 ━━━━━━━━━━━━━━━━━━━━ 273s 4s/step - accuracy: 0.7372 - loss: 0.6113 - val_accuracy: 0.5074 - val_loss: 1.5274
Epoch 5/5
68/68 ━━━━━━━━━━━━━━━━━━━━ 299s 4s/step - accuracy: 0.7845 - loss: 0.4968 - val_accuracy: 0.6342 - val_loss: 2.3302


The training accuracy has improved consistently from 42.88% to 78.45% over the 5 epochs. This indicates that the model is learning effectively from the training data. Validation Accuracy also improves but dusplays some fluctuation, starting at 43.20%, peaking at 63.42% in the last epoch, but dipping in the fourth epoch. This behavior may suggest some issues with generalization or response to model changes per epoch.

On the other hand, the training loss starts unusually high at 3461.5281, which is atypical and might indicate a problem with how the loss is being calculated or an issue with the initial weights. However, it decreases to a more typical value of 0.4968 by the final epoch, showing proper learning progression.
Validation Loss starts at 1.0446 and increases to 2.3302 by the end of the training. The increasing validation loss as training progresses is concerning and might suggest that the model is starting to overfit the training data.


**STEP 5: Evaluate the Model**

In [10]:
# Evaluate the model's performance on the test data
score = CNN_model.evaluate(test_images, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

22/22 ━━━━━━━━━━━━━━━━━━━━ 14s 631ms/step - accuracy: 0.6366 - loss: 3.4305
Test loss: 3.776289463043213
Test accuracy: 0.6088235378265381


### Key Observations


The model's increasing loss on the validation set suggests it may be overfitting the training data. Overfitting happens when a model learns the details and noise in the training data to an extent that it negatively impacts the performance of the model on new data. To combat overfitting, we could increase the diversity of the training data however in this case I will be moving on to a pre-trained model, specifically ResNet-50.


<hr>

In [51]:
model.save("trained_CNN.keras")